# GoudaCell Interactive Segmentation

This notebook allows you to test segmentation parameters on a few images.
Once you find good parameters, it automatically generates a config file for batch processing.

**Requirements:**
- Run this on a GPU node via SLURM (see `scripts/jupyter_gpu.sh`)
- Install goudacell with either `cellpose3` or `cellpose4` extras

In [ ]:
# Check GPU availability
import torch

print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

In [ ]:
# Check goudacell and cellpose versions
import goudacell
from goudacell.segment import get_cellpose_version

print(f"GoudaCell version: {goudacell.__version__}")
cp_version = get_cellpose_version()
print(f"Cellpose version: {cp_version[0]}.{cp_version[1]}")

if cp_version[0] >= 4:
    print("Available models: cpsam")
    DEFAULT_MODEL = "cpsam"
else:
    print("Available models: cyto3, nuclei, cyto2, cyto")
    DEFAULT_MODEL = "cyto3"

---
## 1. Configuration

**Set all your parameters here.** These will be used for testing AND for generating the batch config.

In [ ]:
from pathlib import Path

# ============================================
# EDIT THESE PARAMETERS
# ============================================

# Input/Output paths
# Set TEST_IMAGE to a specific file, or leave as None to auto-select from INPUT_DIR
TEST_IMAGE = None  # e.g., Path("/path/to/specific/image.tif")
INPUT_DIR = Path("../data")  # Directory with images (used if TEST_IMAGE is None)
OUTPUT_DIR = Path("../data/masks")  # Where to save masks
FILE_PATTERN = "*.tif"  # File pattern (*.tif, *.nd2, *.dv)

# Cellpose parameters
MODEL = DEFAULT_MODEL  # "cyto3" for Cellpose 3.x, "cpsam" for 4.x
DIAMETER = 30.0  # Cell diameter in pixels
FLOW_THRESHOLD = 0.4  # Lower = fewer cells (more conservative)
CELLPROB_THRESHOLD = 0.0  # Higher = fewer cells (higher confidence)

# Processing options
GPU = True
REMOVE_EDGE_CELLS = True
Z_PROJECT = True
CHANNEL_TO_SEGMENT = None  # Set to channel index or None for all

# ============================================
print("Configuration set!")
print(f"  Model: {MODEL}")
print(f"  Diameter: {DIAMETER}")

---
## 2. Load Test Image

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from goudacell import load_image

# Resolve test image path
if TEST_IMAGE is not None:
    test_image_path = Path(TEST_IMAGE)
    print(f"Using specified image: {test_image_path}")
else:
    available_files = sorted(INPUT_DIR.glob(FILE_PATTERN))
    if not available_files:
        raise FileNotFoundError(f"No files matching '{FILE_PATTERN}' in {INPUT_DIR}")
    test_image_path = available_files[0]
    print(f"Auto-selected: {test_image_path.name}")
    print(f"Total files in directory: {len(available_files)}")

# Load image
image = load_image(test_image_path, channel=CHANNEL_TO_SEGMENT, z_project=Z_PROJECT)
print(f"Image shape: {image.shape}")
print(f"Image dtype: {image.dtype}")

In [ ]:
# Visualize the image
fig, axes = plt.subplots(1, min(image.shape[0], 4) if image.ndim == 3 else 1, figsize=(16, 4))

if image.ndim == 2:
    axes.imshow(image, cmap="gray")
    axes.set_title("Image")
    axes.axis("off")
else:
    if image.shape[0] == 1:
        axes = [axes]
    for i, ax in enumerate(axes if hasattr(axes, "__iter__") else [axes]):
        if i < image.shape[0]:
            ax.imshow(image[i], cmap="gray")
            ax.set_title(f"Channel {i}")
        ax.axis("off")

plt.tight_layout()
plt.show()

---
## 3. Estimate Diameter (Cellpose 3.x only)

In [ ]:
from goudacell.segment import estimate_diameter, get_cellpose_version

if get_cellpose_version()[0] < 4:
    try:
        estimated = estimate_diameter(image, model=MODEL, gpu=GPU)
        print(f"Estimated cell diameter: {estimated:.1f} pixels")
        print(f"Current DIAMETER setting: {DIAMETER}")
        print("\nUpdate DIAMETER in Section 1 if needed, then re-run from there.")
    except Exception as e:
        print(f"Could not estimate diameter: {e}")
else:
    print("Automatic diameter estimation not available in Cellpose 4.x")
    print(f"Current DIAMETER setting: {DIAMETER}")
    print("Adjust manually based on your images.")

---
## 4. Run Segmentation

In [ ]:
from goudacell import segment

# Run segmentation with current parameters
masks = segment(
    image,
    diameter=DIAMETER,
    model=MODEL,
    flow_threshold=FLOW_THRESHOLD,
    cellprob_threshold=CELLPROB_THRESHOLD,
    gpu=GPU,
    remove_edge_cells=REMOVE_EDGE_CELLS,
)

n_cells = len(np.unique(masks)) - 1  # Exclude background
print(f"Found {n_cells} cells")

In [ ]:
# Visualize segmentation
from matplotlib.colors import ListedColormap

# Create random colormap for masks
n_labels = masks.max() + 1
colors = np.random.rand(n_labels, 4)
colors[0] = [0, 0, 0, 0]  # Background transparent
cmap = ListedColormap(colors)

# Get display image
if image.ndim == 2:
    display_img = image
else:
    display_img = image[0] if image.shape[0] == 1 else np.max(image, axis=0)

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

axes[0].imshow(display_img, cmap="gray")
axes[0].set_title("Original")
axes[0].axis("off")

axes[1].imshow(masks, cmap=cmap)
axes[1].set_title(f"Segmentation ({n_cells} cells)")
axes[1].axis("off")

axes[2].imshow(display_img, cmap="gray")
axes[2].imshow(masks, cmap=cmap, alpha=0.4)
axes[2].set_title("Overlay")
axes[2].axis("off")

plt.tight_layout()
plt.show()

---
## 5. Parameter Sweep (Optional)

Test different parameter values to find optimal settings.

In [ ]:
# Test different diameters
test_diameters = [20, 30, 40, 50]

fig, axes = plt.subplots(1, len(test_diameters), figsize=(4 * len(test_diameters), 4))

for i, d in enumerate(test_diameters):
    masks_test = segment(image, diameter=d, model=MODEL, gpu=GPU)
    n = len(np.unique(masks_test)) - 1

    axes[i].imshow(display_img, cmap="gray")
    axes[i].imshow(masks_test, cmap=cmap, alpha=0.4)
    axes[i].set_title(f"d={d} ({n} cells)")
    axes[i].axis("off")

plt.suptitle("Diameter comparison")
plt.tight_layout()
plt.show()

In [ ]:
# Test different flow thresholds
test_flows = [0.2, 0.4, 0.6, 0.8]

fig, axes = plt.subplots(1, len(test_flows), figsize=(4 * len(test_flows), 4))

for i, ft in enumerate(test_flows):
    masks_test = segment(image, diameter=DIAMETER, model=MODEL, flow_threshold=ft, gpu=GPU)
    n = len(np.unique(masks_test)) - 1

    axes[i].imshow(display_img, cmap="gray")
    axes[i].imshow(masks_test, cmap=cmap, alpha=0.4)
    axes[i].set_title(f"flow={ft} ({n} cells)")
    axes[i].axis("off")

plt.suptitle("Flow threshold comparison")
plt.tight_layout()
plt.show()

---
## 6. Generate Batch Config

**Once you're happy with the parameters above, run this cell to generate the config file.**

No manual editing required - the config uses all the parameters from Section 1.

In [ ]:
from goudacell.config import SegmentationConfig

# Generate config from current parameters
config = SegmentationConfig(
    input_dir=str(INPUT_DIR),
    output_dir=str(OUTPUT_DIR),
    file_pattern=FILE_PATTERN,
    model=MODEL,
    diameter=DIAMETER,
    channels=[0, 0],  # Adjust if using multi-channel
    flow_threshold=FLOW_THRESHOLD,
    cellprob_threshold=CELLPROB_THRESHOLD,
    gpu=GPU,
    remove_edge_cells=REMOVE_EDGE_CELLS,
    z_project=Z_PROJECT,
    channel_to_segment=CHANNEL_TO_SEGMENT,
)

# Save config
config_path = Path(OUTPUT_DIR).parent / "segmentation_config.yaml"
config.to_yaml(config_path)

print(f"Config saved to: {config_path}")
print()
print("=" * 50)
print("To run batch segmentation:")
print("=" * 50)
print(f"sbatch scripts/run_segmentation.sh {config_path}")
print()
print("Or run directly:")
print(f"goudacell segment {config_path}")

In [ ]:
# Preview the generated config
print("Generated config:")
print("=" * 50)
with open(config_path) as f:
    print(f.read())

---
## 7. Save Test Results (Optional)

In [ ]:
from goudacell import save_mask

# Save the test segmentation mask
test_output = OUTPUT_DIR / f"{test_image_path.stem}_mask.tif"
save_mask(masks, test_output)
print(f"Saved test mask to: {test_output}")